Lab Assignment 2: Data Reading and Processing
Name: Diego T. 
Course: CS 456
Date: Oct, 13, 2025

Introduction: 
This lab focuses on reading, cleaning, and combining multiple data files using Python and Pandas. The provided datasets are partially "damaged," containing missing values and formatting issues that must be identified and corrected. The main goal is to integrate all files into a single, well-structured dataset and analyze it to find useful insights, such as total data volume, missing data, and company performance from 1995 to 1998. This project also emphasizes clear documentation and organized code to ensure the data processing steps are easy to follow and reproducible. 

We begin by importing libraries that we will use.

In [4]:
# import pandas for working with data tables
import pandas as pd

# import numpy for handling missing or numeric values
import numpy as np

# import re for text pattern matching (used later for text file cleanup)
import re

# json for safely loading JSON objects
import json

Step 1 - Reading and Combining the Data
In this section, all three data files are imported and combined into one dataset. Each file uses a different format (CSV, JSON, and text), so different loading methods are used. After reading the data, all column names are standardized and merged into a single DataFrame.

In [7]:
# read the CSV file that contains Fortune 500 data
# some rows might have extra commas, so we skip those instead of crashing
csv_data = pd.read_csv("fortune500.csv", on_bad_lines='skip', engine='python')
print("CSV shape:", csv_data.shape)

# read the JSON file line by line since each line is one JSON object
json_records = []   # will store each valid JSON record we find
with open("lines.json", "r") as jfile:
    for line in jfile:
        line = line.strip()        # remove spaces and newline characters
        if not line:               # skip empty lines to avoid errors
            continue
        try:
            data = json.loads(line)          # convert line into a dictionary
            json_records.append(data)        # add valid line to the list
        except json.JSONDecodeError:
            continue                         # skip bad or incomplete lines

json_data = pd.DataFrame(json_records)       # convert list into a DataFrame
print("JSON shape:", json_data.shape)

# read the unstructured text file and extract key-value pairs manually
records = []      # list for all records
temp = {}         # temporary dictionary for one record
with open("unstructureddata.txt", "r") as f:
    for line in f:
        line = line.strip()                  # remove spaces and newlines
        if not line:                         # blank line marks end of one record
            if temp:                         # only add if dictionary not empty
                records.append(temp)
                temp = {}                    # reset for next record
        else:
            if ":" in line:                  # split line if it has a key and value
                key, value = line.split(":", 1)
                temp[key.strip()] = value.strip()
    if temp:                                 # add last record if needed
        records.append(temp)

txt_data = pd.DataFrame(records)             # make DataFrame from text records
print("TXT shape:", txt_data.shape)

# combine all three datasets into one large DataFrame
combined = pd.concat([csv_data, json_data, txt_data], ignore_index=True)
print("Combined shape:", combined.shape)

# preview the first few rows to confirm that data loaded correctly
combined.head()

CSV shape: (28851, 5)
JSON shape: (10874, 5)
TXT shape: (7600, 5)
Combined shape: (47325, 10)


,#Year,(1)Rank,!Company,(3)Revenue (in millions),okjb)Profit (in millions),Year,Rank,Company,Revenue (in millions),Profit (in millions)
0,1955.0,1,General Motors,9823.5,806,NaN,NaN,NaN,NaN,NaN
1,1955.0,2,Exxon Mobil,5661.4,584.8,NaN,NaN,NaN,NaN,NaN
2,1955.0,3,U.S. Steel,3250.4,195.4,NaN,NaN,NaN,NaN,NaN
3,1955.0,4,General Electric,NaN,212.6,NaN,NaN,NaN,NaN,NaN
4,1955.0,5,Esmark,2510.8,19.1,NaN,NaN,NaN,NaN,NaN


Step 2 - Cleaning the Data
Here I fixed column names, removed duplicates, and made sure numbers were in the right format. I also checked for missing values and saved the cleaned version as a new CSV file. 

In [8]:
# start by checking what columns exist and how they look
print("Original column names:")
print(list(combined.columns))
print()

# make all column names lowercase, remove spaces, and replace them with underscores
combined.columns = [
    c.strip().lower().replace(" ", "_").replace("(in_millions)", "")
    for c in combined.columns
]

# check again to confirm column names were cleaned correctly
print("Cleaned column names:")
print(list(combined.columns))
print()

# remove any duplicate rows that might exist after merging multiple files
combined = combined.drop_duplicates()

# loop through columns and convert numeric-type columns to numbers when possible
for name in combined.columns:
    if any(word in name for word in ["year", "rank", "revenue", "profit"]):
        # convert non-numeric values to NaN instead of causing an error
        combined[name] = pd.to_numeric(combined[name], errors="coerce")

# replace blank strings in company names with NaN (so they can be dropped)
combined["company"] = combined["company"].replace("", np.nan)

# drop any rows where the company name is missing (since it's the main identifier)
combined = combined.dropna(subset=["company"])

# print total missing values per column so we know data quality
print("Missing values per column:")
print(combined.isnull().sum())
print()

# fill obvious missing numeric values with NaN (no change, but ensures consistency)
combined = combined.fillna(value=np.nan)

# display dataset shape after cleaning
print("Cleaned dataset shape:", combined.shape)

# save the cleaned dataset to a new CSV file
combined.to_csv("clean_combined.csv", index=False)

# confirmation message so we know the save worked
print("Cleaned dataset saved as clean_combined.csv")

Original column names:
['#Year', '(1)Rank', '!Company', '(3)Revenue (in millions)', 'okjb)Profit (in millions)', 'Year', 'Rank', 'Company', 'Revenue (in millions)', 'Profit (in millions)']

Cleaned column names:
['#year', '(1)rank', '!company', '(3)revenue_', 'okjb)profit_', 'year', 'rank', 'company', 'revenue_', 'profit_']

Missing values per column:
#year           18471
(1)rank         18471
!company        18471
(3)revenue_     18471
okjb)profit_    18471
year                0
rank               11
company             0
revenue_         2004
profit_           744
dtype: int64

Cleaned dataset shape: (18471, 10)
Cleaned dataset saved as clean_combined.csv


/var/folders/f8/g51pfbbs31x4tnfh4q3bbp9c0000gn/T/ipykernel_28367/4075144424.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined = combined.fillna(value=np.nan)


Step 3 - Analyzing the Data
Once everything is clean, I calculated a few key stats: total number of records, total missing data points, how many unique companies, which companies made the most revenue and profit between 1995-1998.
All the results are stored in a small summary table. 

In [12]:
# find total number of rows in the dataset
total_records = len(combined)

# count total number of missing cells across the entire dataset
missing_cells = combined.isnull().sum().sum()

# count how many unique companies exist
unique_companies = combined["company"].nunique()

# look for columns that might represent revenue or profit
revenue_cols = [col for col in combined.columns if isinstance(col, str) and ("revenue" in col.lower() or "sales" in col.lower())]
profit_cols = [col for col in combined.columns if isinstance(col, str) and ("profit" in col.lower() or "income" in col.lower())]

# make sure we found at least one valid column name for each
revenue_col = revenue_cols[0] if revenue_cols else None
profit_col = profit_cols[0] if profit_cols else None

# print detected column names so we can confirm visually
print("Detected revenue column:", revenue_col)
print("Detected profit column:", profit_col)
print()

# make a subset for years between 1995 and 1998 if the 'year' column exists
if "year" in combined.columns:
    subset = combined[(combined["year"] >= 1995) & (combined["year"] <= 1998)]
else:
    subset = combined.copy()  # fallback if 'year' is missing

# initialize placeholder results in case we can't compute them
top_revenue_company = "N/A"
top_profit_company = "N/A"

# only run if we found both columns and the subset isn't empty
if not subset.empty and revenue_col and profit_col:
    # drop rows where revenue or profit are NaN before finding max
    subset_valid = subset.dropna(subset=[revenue_col, profit_col], how='any')
    
    if not subset_valid.empty:
        # safely get top revenue company
        try:
            top_revenue_company = subset_valid.loc[subset_valid[revenue_col].idxmax(), "company"]
        except Exception:
            top_revenue_company = "N/A"
        # safely get top profit company
        try:
            top_profit_company = subset_valid.loc[subset_valid[profit_col].idxmax(), "company"]
        except Exception:
            top_profit_company = "N/A"

# create a dictionary to hold all summary results
results_dict = {
    "total_records": [total_records],
    "missing_cells": [missing_cells],
    "unique_companies": [unique_companies],
    "top_revenue_1995_1998": [top_revenue_company],
    "top_profit_1995_1998": [top_profit_company]
}

# make a new DataFrame to display results nicely
results_df = pd.DataFrame(results_dict)

# show results table in the output
print("Summary of results:")
print(results_df)
print()

# save the summary to a CSV file
results_df.to_csv("Results_Combine.csv", index=False)

# print confirmation message
print("Results_Combine.csv has been created and saved successfully.")


Detected revenue column: (3)revenue_
Detected profit column: okjb)profit_

Summary of results:
   total_records  missing_cells  unique_companies top_revenue_1995_1998  \
0          18471          95114              2226                   N/A   

  top_profit_1995_1998  
0                  N/A  

Results_Combine.csv has been created and saved successfully.


After cleaning and merging all three datasets, the final combined file contains consistent company information across multiple years. The analysis produced the total record count, number of missing values, and identified the top companies by revenue and profit between 1995 and 1998. The cleaned dataset ('clean_combined.csv') and the analysis results ('Results_Combine.csv') were both saved successfully.